# Non-parametric models

Here we describe the `PySM` interface to models that can not be described parametrically. `PySM` allows for a set of emission templates at different frequencies to be provided, and then interpolates between these templates in each pixel. 

## Interface

In order to implement your model in `PySM` in this way:

1. Provide a data directory containing a set of $N$ fits files at frequencies $\nu_i$ where $i = 1, 2, ..., N$, each containing the model $[T, Q, U][\nu_i]$ maps. 
2. In this directory provide a file in with two columns:
    - Column 1: each row is frequency $\nu_i$ in GHz. 
    - Column 2: each row is the path to the file containing the $(T, Q, U)[\nu_i]$. 
3. In [`pysm.nominal`](pysm/nominal.py) add a model dictionary with `interpolation=True` and `info_file='path/to/info/file'`, e.g.: 
```python
    def dust_interpolation(nside, pixel_indices=None):
        return {
            'interpolation': True,
            'info_file': '/absolute/path/to/file/with/info',
        }
```
4. You can now call this model in the same way as other components: 
```python
    import pysm
    from pysm.nominal import models
    
    def main():
        # Specify the model just created. 
        sky_config = {'dust': models('dust_interpolation', nside)}
        sky = pysm.Sky(sky_config)
        return
```

## Implementation

`PySM` reads in the $3N$ maps, and does a cubic spline interpolation between freuqencies, in each pixel. The interpolation uses the [`scipy.interpolate.CubicSpline`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.interpolate.CubicSpline.html) routine, with `extrapolation=True`. This means that if the spline is queried at $\nu>\nu_N$ or $\nu<\nu_0$, it will not fail, but will extrapolate the data provided. It is the user's responsibility to ensure that the model provided is densely sampled enough in frequency, and covers a broad enough range of frequencies, that this implementation does not produce undesired results. 

## Example

We now provide an example of implementing a model in this way, from scratch. 

In order to generate the $N$ samples let's just use a simple model:


In [27]:
import numpy as np
import healpy as hp
import os

def power_law(nu, nu_0, beta):
    return (nu / nu_0) ** beta 

n_maps = 20 # number of samples to generate
nu_0 = 353. # reference frequency in model
nu_min = 100 # minimum frequency to include
nu_max = 400 # maximum frequency to include
nside = 64 # nside at which to generate samples
npix = hp.nside2npix(nside)
# Make array of frequencies
nus = np.linspace(nu_min, nu_max, n_maps)
# Make spectral index maps
index = 1.5 + 0.05 * np.random.randn(3 * npix).reshape((3, npix))
# Make sample maps
maps = power_law(nus.reshape(len(nus), 1, 1), nu_0, index)
# Make paths in which to store these sample maps
data_dir = os.path.abspath(os.path.dirname('.'))
fpaths = [os.path.join(data_dir, 'map{:03d}.fits'.format(i)) for i in range(len(maps))]
# Save the sample maps
for fpath, hpix_map in zip(fpaths, maps):
    hp.write_map(fpath, hpix_map, overwrite=True)
# Make an example info file that will be given to PySM with paths to the data. 
dat = np.array(zip(nus, fpaths), dtype=[('nus', float), ('paths', object)])
np.savetxt('test.txt', dat, delimiter=" ", fmt="%.4f %s")

['/home/bthorne/Projects/PySM/PySM_public/map000.fits', '/home/bthorne/Projects/PySM/PySM_public/map001.fits', '/home/bthorne/Projects/PySM/PySM_public/map002.fits', '/home/bthorne/Projects/PySM/PySM_public/map003.fits', '/home/bthorne/Projects/PySM/PySM_public/map004.fits', '/home/bthorne/Projects/PySM/PySM_public/map005.fits', '/home/bthorne/Projects/PySM/PySM_public/map006.fits', '/home/bthorne/Projects/PySM/PySM_public/map007.fits', '/home/bthorne/Projects/PySM/PySM_public/map008.fits', '/home/bthorne/Projects/PySM/PySM_public/map009.fits', '/home/bthorne/Projects/PySM/PySM_public/map010.fits', '/home/bthorne/Projects/PySM/PySM_public/map011.fits', '/home/bthorne/Projects/PySM/PySM_public/map012.fits', '/home/bthorne/Projects/PySM/PySM_public/map013.fits', '/home/bthorne/Projects/PySM/PySM_public/map014.fits', '/home/bthorne/Projects/PySM/PySM_public/map015.fits', '/home/bthorne/Projects/PySM/PySM_public/map016.fits', '/home/bthorne/Projects/PySM/PySM_public/map017.fits', '/home/bt

The contents of the info file are:

In [25]:
!cat 'test.txt'

100.0000 /home/bthorne/Projects/PySM/PySM_public/map000.fits
115.7895 /home/bthorne/Projects/PySM/PySM_public/map001.fits
131.5789 /home/bthorne/Projects/PySM/PySM_public/map002.fits
147.3684 /home/bthorne/Projects/PySM/PySM_public/map003.fits
163.1579 /home/bthorne/Projects/PySM/PySM_public/map004.fits
178.9474 /home/bthorne/Projects/PySM/PySM_public/map005.fits
194.7368 /home/bthorne/Projects/PySM/PySM_public/map006.fits
210.5263 /home/bthorne/Projects/PySM/PySM_public/map007.fits
226.3158 /home/bthorne/Projects/PySM/PySM_public/map008.fits
242.1053 /home/bthorne/Projects/PySM/PySM_public/map009.fits
257.8947 /home/bthorne/Projects/PySM/PySM_public/map010.fits
273.6842 /home/bthorne/Projects/PySM/PySM_public/map011.fits
289.4737 /home/bthorne/Projects/PySM/PySM_public/map012.fits
305.2632 /home/bthorne/Projects/PySM/PySM_public/map013.fits
321.0526 /home/bthorne/Projects/PySM/PySM_public/map014.fits
336.8421 /home/bthorne/Projects/PySM/PySM_public/map015.fits
352.6316 /home/bthorne/P

Let's then make the new model dictionary that this would correspond to. In practice we would need to be more careful about paths, because this dictionary would probably live in `pysm.nominal`. But here we are just defining the dictionary in this notebook.

In [28]:
interp_model = {
    'interpolation': True,
    'info_file': 'test.txt',
}

In [29]:
import pysm
sky_config = {'dust': interp_model}
sky = pysm.Sky(sky_config)

AttributeError: 'str' object has no attribute 'keys'

In [ ]:
# Remove maps
for fpath, hpix_map in zip(fpaths, maps):
    os.remove(fpath)
# Remove info file
os.remove('test.txt')